In [281]:
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import psycopg2
from collections import OrderedDict

In [282]:
# To compute downselection potential of each inclusion criteria primitive, we have to profile the dataset we'll run against.
# For expediency, we can sample many times from the MIMIC dataset, and get an average of 0s/1s for each
# We can also consider using the structured data

# Our 27 inclusion criteria primitives are: 
# 1. Gender: ["male", "female"]; cardinality = 2
# 2. Race: ["asian", "black", "native_american", "pacific_islander", "white"]; cardinality = 5
# 3. Condition(s): ["hypertension", "chf", "afib", "diabetes", "renal_failure", "high_cholesterol", "uti", "gerd", "arteriosclerosis", "respiratory_failure"]; high-level cardinality = 10 (individual terms may be expanded to catch clinically equivalent synonyms)
# 4. Medication(s): []; high-level cardinality = 10 (individual terms may be expanded to catch clinically equivalent synonyms)


In [283]:
import psycopg2
from config import config
 
def connect():
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # read connection parameters
        params = config()
 
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params)
 
        # create a cursor
        cur = conn.cursor()
        
 # execute a statement
        print('PostgreSQL database version:')
        cur.execute('SELECT version()')
 
        # display the PostgreSQL database server version
        db_version = cur.fetchone()
        print(db_version)
       
     # close the communication with the PostgreSQL
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
            print('Database connection closed.')
            
def run_query(query_str, primitive, category=None):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # read connection parameters
        params = config()
 
        # connect to the PostgreSQL server
        #print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params)
 
        # create a cursor
        cur = conn.cursor()
        
 # execute a statement
        #print('Primitive" {}; category: {}'.format(primitive, category))
        cur.execute(query_str)
 
        # display the PostgreSQL database server version
        #db_version = cur.fetchone()
        results = cur.fetchall()
        #print(db_version)
        
        #print("Number of unique positive instances: {}".format(results[0][0]))
     # close the communication with the PostgreSQL
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
            #print('Database connection closed.')
            
    return results[0][0]
            
    
    
    
 
 
if __name__ == '__main__':
    connect()

Connecting to the PostgreSQL database...
PostgreSQL database version:
('PostgreSQL 10.0 on x86_64-pc-linux-gnu, compiled by gcc (Debian 6.3.0-18) 6.3.0 20170516, 64-bit',)
Database connection closed.


In [284]:
def generate_primitives_dict():

    primitives = OrderedDict()
    primitives["gender"] = {"table":"person", "column":"gender_source_value", "count_distinct":"person_id", "results":[],
                            "categories":["male", "female"], "sql_values":[["M"], ["F"]]}

    primitives["gender"]["results"] = [[0] for x in primitives["gender"]["categories"]]
    primitives["gender"]["percent_eliminated"] = [[0] for x in primitives["gender"]["categories"]]

    primitives["race"] = {"table":"person", "column":"race_concept_id", "count_distinct":"person_id", "results":[],
                          "categories":["asian", "black", "native_american", "pacific_islander", "white"],
                          "sql_values":[[8515], [8516], [8657], [84557], [8527]]}

    primitives["race"]["results"] = [[0] for x in primitives["race"]["categories"]]
    primitives["race"]["percent_eliminated"] = [[0] for x in primitives["race"]["categories"]]

    '''
    SELECT distinct(concept_name), condition_concept_id FROM
    /*SELECT distinct( condition_concept_id) FROM*/

    (SELECT distinct(concept_id) as unique_concept_ids, concept_name
    FROM mimic_v5.concept
    WHERE lower(concept_name) like '%TERM%'
    ) a

    JOIN

    (SELECT condition_concept_id from mimic_v5.condition_occurrence) b

    ON a.unique_concept_ids = b.condition_concept_id;'''



    ##### CONDITIONS #####
    #%% Hypertension (high blood pressure returns empty set)
    # Benign essential hypertension | 312648
    # Benign intracranial hypertension | 312902
    # Benign secondary renovascular hypertension | 4249016
    # Essential hypertension | 320128
    # Heritable pulmonary arterial hypertension | 44783618
    # Malignant essential hypertension | 317898
    # Malignant secondary renovascular hypertension | 4110947
    # Portal hypertension | 192680
    # Renal disease in pregnancy AND/OR puerperium without hypertension | 192679
    # Renovascular hypertension | 317895
    # Secondary hypertension | 319826

    #%% congestive heart failure
    # Benign hypertensive heart disease with congestive heart failure	312338
    # Benign hypertensive heart disease without congestive heart failure	313502
    # Hypertensive heart disease with congestive heart failure	314378
    # Hypertensive heart disease without congestive heart failure	319034
    # Malignant hypertensive heart disease with congestive heart failure	316994
    # Malignant hypertensive heart disease without congestive heart failure	314369

    #%% afib
    # Atrial flutter	314665
    # Ventricular flutter	433225
    # Atrial fibrillation=313217
    # Ventricular fibrillation	437894

    #%% diabetes
    # Coma associated with diabetes mellitus	443735
    # Diabetes insipidus	30968
    # Diabetes mellitus during pregnancy - baby delivered	443012
    # Diabetes mellitus during pregnancy - baby not yet delivered	192691
    # Diabetes mellitus in the puerperium - baby delivered during previous episode of care	4062686
    # Diabetic oculopathy associated with type 1 diabetes mellitus	373999
    # Diabetic oculopathy associated with type 2 diabetes mellitus	443733
    # Disorder due to type 1 diabetes mellitus	435216
    # Disorder due to type 2 diabetes mellitus	443732
    # Hyperosmolality due to uncontrolled type 1 diabetes mellitus	443592
    # Hypoglycemic coma in type 1 diabetes mellitus	4228112
    # Ketoacidosis in type 1 diabetes mellitus	439770
    # Ketoacidosis in type 2 diabetes mellitus	443734
    # Neonatal diabetes mellitus	193323
    # Nephrogenic diabetes insipidus	438476
    # Neurological disorder associated with type 1 diabetes mellitus	377821
    # Neurologic disorder associated with diabetes mellitus	443730
    # Neurologic disorder associated with type 2 diabetes mellitus	376065
    # Peripheral circulatory disorder associated with type 1 diabetes mellitus	318712
    # Peripheral circulatory disorder associated with type 2 diabetes mellitus	443729
    # Renal disorder associated with type 1 diabetes mellitus	200687
    # Renal disorder due to type 2 diabetes mellitus	443731
    # Secondary diabetes mellitus	195771
    # Type 1 diabetes mellitus	201254
    # Type 1 diabetes mellitus uncontrolled	40484648
    # Type 1 diabetes mellitus with hyperosmolar coma	201531
    # Type 2 diabetes mellitus	201826
    # Type 2 diabetes mellitus with hyperosmolar coma	201530
    # Type II diabetes mellitus uncontrolled	40482801


    #%%renal failure
    # Acute renal failure due to acute cortical necrosis	197329
    # Acute renal failure syndrome	197320
    # Acute renal papillary necrosis with renal failure	432961
    # Anemia of chronic renal failure	443961
    # Hypertensive heart and renal disease with both (congestive) heart failure and renal failure	439694
    # Hypertensive heart and renal disease with renal failure	439695
    # Hypertensive renal disease with renal failure	439697
    # Post-delivery acute renal failure - delivered with postnatal problem	196764
    # Post-delivery acute renal failure with postnatal problem	4066005
    # Renal failure syndrome	192359

    #%% high cholesterol
    # Hyperlipidemia	432867
    # Mixed hyperlipidemia	438720
    # Pure hypercholesterolemia	437827

    #%% uti
    # Genitourinary tract infection in pregnancy - delivered	81357
    # Genitourinary tract infection in pregnancy - delivered with postnatal complication	80778
    # Genitourinary tract infection in pregnancy - not delivered	74415
    # Neonatal urinary tract infection	4047937
    # Urinary tract infectious disease	81902


    #%% gerd
    # Gastroesophageal reflux disease	318800

    #%% arteriosclerosis
    # Arteriosclerosis of arterial coronary artery bypass graft	40483189
    # Arteriosclerosis of artery of extremity	40484167
    # Arteriosclerosis of autologous vein coronary artery bypass graft	40482638
    # Arteriosclerosis of coronary artery bypass graft	443563
    # Arteriosclerosis of nonautologous coronary artery bypass graft	40482655
    # Coronary arteriosclerosis	317576
    # Coronary arteriosclerosis in native artery	42872402

    #%% respiratory failure
    # Acute-on-chronic respiratory failure	312940
    # Acute respiratory failure	319049
    # Chronic respiratory failure	314971
    # Perinatal respiratory failure	313590


    primitives["conditions"] = {"table":"condition_occurrence", "column":"condition_concept_id", "count_distinct":"condition_occurrence_id", 
                          "categories":["hypertension", "chf", "afib", "diabetes", "renal_failure", "high_cholesterol",
                                       "uti", "gerd", "arteriosclerosis", "respiratory_failure"],
                          "sql_values":[[312648, 312902, 4249016, 320128, 44783618,  317898, 4110947, 192680, 192679, 317895, 319826], 
                                        [312338, 313502, 314378, 319034, 319034, 316994, 314369],
                                        [314665, 433225, 313217, 437894], 
                                        [443731, 4228112, 40484648, 435216, 443732, 193323, 443733, 443734, 201531, 30968, 40482801, 4062686, 318712, 192691, 200687, 376065, 439770, 438476, 195771, 443592, 443730, 443012, 443729, 377821, 373999, 201254, 201530, 443735, 201826], 
                                        [439695, 192359, 439694, 432961, 196764, 443961, 4066005, 439697, 197320, 197329],
                                        [432867,438720, 437827], 
                                        [81357, 80778, 74415, 4047937, 81902],
                                        [318800],
                                        [40483189,40484167, 40482638, 443563, 40482655, 317576, 42872402], 
                                        [312940, 319049, 314971, 313590]]}


    primitives["conditions"]["results"] = [[0] for x in primitives["conditions"]["categories"]]
    primitives["conditions"]["percent_eliminated"] = [[0] for x in primitives["conditions"]["categories"]]

    '''

    SELECT distinct(concept_name), drug_concept_id FROM

    (SELECT distinct(concept_id) as unique_concept_ids, concept_name
    FROM mimic_v5.concept
    WHERE lower(concept_name) like '%TERM%'
    ) a

    JOIN

    (SELECT drug_concept_id from mimic_v5.drug_exposure) b

    ON a.unique_concept_ids = b.drug_concept_id;'''



    ##### MEDICATIONS #####

    #%% sodium chloride

    # Acetate 0.044 MEQ/ML / Cysteine 0.2 MG/ML / Histidine 2.5 MG/ML / Isoleucine 5.6 MG/ML / Leucine 8.8 MG/ML / Lysine 9 MG/ML / Methionine 8.8 MG/ML / Phenylalanine 8.8 MG/ML / sodium bisulfite 0.5 MG/ML / Sodium Chloride 0.005 MEQ/ML / Threonine 4 MG/ML /	19131053
    # Ascorbic Acid 4.7 MG/ML / POLYETHYLENE GLYCOL 3350 100 MG/ML / Potassium Chloride 0.0136 MEQ/ML / Sodium Ascorbate 5.9 MG/ML / Sodium Chloride 0.046 MEQ/ML / sodium sulfate 7.5 MG/ML Oral Solution [MoviPrep]	19131009
    # Calcium Chloride 0.0014 MEQ/ML / Glucose 50 MG/ML / Potassium Chloride 0.024 MEQ/ML / Sodium Chloride 0.103 MEQ/ML / Sodium Lactate 0.028 MEQ/ML Injectable Solution	968984
    # Calcium Chloride 0.0014 MEQ/ML / Potassium Chloride 0.004 MEQ/ML / Sodium Chloride 0.103 MEQ/ML / Sodium Lactate 0.028 MEQ/ML Injectable Solution	19135374
    # Calcium Chloride 0.001 MEQ/ML / Glucose 50 MG/ML / Potassium Chloride 0.004 MEQ/ML / Sodium Chloride 0.103 MEQ/ML / Sodium Lactate 0.028 MEQ/ML Injectable Solution	968985
    # Calcium Chloride 0.004 MEQ/ML / Magnesium Chloride 0.003 MEQ/ML / Potassium Chloride 0.01 MEQ/ML / Sodium Acetate 0.0475 MEQ/ML / Sodium Chloride 0.109 MEQ/ML / sodium citrate 0.012 MEQ/ML Ophthalmic Solution [BSS Ophthalmic Solution]	19132042
    # Glucose 50 MG/ML / Potassium Chloride 0.02 MEQ/ML / Sodium Chloride 0.0769 MEQ/ML Injectable Solution	19123351
    # Glucose 50 MG/ML / Potassium Chloride 0.02 MEQ/ML / Sodium Chloride 0.154 MEQ/ML Injectable Solution	19123359
    # Glucose 50 MG/ML / Potassium Chloride 0.04 MEQ/ML / Sodium Chloride 0.0769 MEQ/ML Injectable Solution	19123353
    # Glucose 50 MG/ML / Potassium Chloride 0.04 MEQ/ML / Sodium Chloride 0.154 MEQ/ML Injectable Solution	968274
    # Glucose 50 MG/ML / Sodium Chloride 0.0342 MEQ/ML Injectable Solution	19124112
    # Glucose 50 MG/ML / Sodium Chloride 0.0769 MEQ/ML Injectable Solution	19076330
    # Glucose 50 MG/ML / Sodium Chloride 0.154 MEQ/ML Injectable Solution	967825
    # Magnesium Chloride 0.00148 MEQ/ML / Potassium Chloride 0.00497 MEQ/ML / Sodium Acetate 0.027 MEQ/ML / Sodium Chloride 0.0899 MEQ/ML / Sodium gluconate 5.02 MG/ML Injectable Solution [Plasmalyte A]	19131118
    # POLYETHYLENE GLYCOL 3350 59 MG/ML / Potassium Chloride 0.01 MEQ/ML / Sodium Bicarbonate 0.02 MEQ/ML / Sodium Chloride 0.025 MEQ/ML / sodium sulfate 0.04 MEQ/ML Oral Solution [Golytely]	40175291
    # Potassium Chloride 0.02 MEQ/ML / Sodium Chloride 0.154 MEQ/ML Injectable Solution	968304
    # Sodium Chloride 0.0769 MEQ/ML Injectable Solution	967905
    # Sodium Chloride 0.154 MEQ/ML Injectable Solution	19079524
    # Sodium Chloride 0.154 MEQ/ML Irrigation Solution	968205
    # Sodium Chloride 0.513 MEQ/ML Inhalant Solution	19127005
    # Sodium Chloride 0.513 MEQ/ML Injectable Solution	19127417
    # Sodium Chloride 0.854 MEQ/ML Ophthalmic Solution [Muro 128]	19128117
    # Sodium Chloride 1000 MG Oral Tablet	19079527
    # Sodium Chloride 4 MEQ/ML Injectable Solution	19079551

    #%% glucose
    # 50 ML Glucose 500 MG/ML Prefilled Syringe	1560751
    # Alanine 10.4 MG/ML / Arginine 5.75 MG/ML / Calcium Chloride 0.004 MEQ/ML / Dibasic potassium phosphate 2.61 MG/ML / Glucose 150 MG/ML / Glycine 5.15 MG/ML / Histidine 2.4 MG/ML / Isoleucine 3 MG/ML / Leucine 3.65 MG/ML / Lysine 2.9 MG/ML / Magnesium Chlor	19131082
    # Alanine 8.8 MG/ML / Arginine 4.89 MG/ML / Calcium Chloride 0.004 MEQ/ML / Dibasic potassium phosphate 2.61 MG/ML / Glucose 50 MG/ML / Glycine 4.38 MG/ML / Histidine 2.04 MG/ML / Isoleucine 2.55 MG/ML / Leucine 3.11 MG/ML / Lysine 2.47 MG/ML / Magnesium Ch	19131065
    # Calcium Chloride 0.0014 MEQ/ML / Glucose 50 MG/ML / Potassium Chloride 0.024 MEQ/ML / Sodium Chloride 0.103 MEQ/ML / Sodium Lactate 0.028 MEQ/ML Injectable Solution	968984
    # Calcium Chloride 0.001 MEQ/ML / Glucose 50 MG/ML / Potassium Chloride 0.004 MEQ/ML / Sodium Chloride 0.103 MEQ/ML / Sodium Lactate 0.028 MEQ/ML Injectable Solution	968985
    # Citric Acid 0.038 MEQ/ML / Glucose 24.5 MG/ML / sodium citrate 0.0748 MEQ/ML Injectable Solution	43531812
    # Ethanol 0.1 ML/ML / Glucose 50 MG/ML Injectable Solution	19096125
    # Glucose 100 MG/ML Injectable Solution	1560553
    # Glucose 200 MG/ML Injectable Solution	1560554
    # Glucose 5000 MG Chewable Tablet	19077654
    # Glucose 50 MG/ML Injectable Solution	19076324
    # Glucose 50 MG/ML / Potassium Chloride 0.02 MEQ/ML Injectable Solution	19076326
    # Glucose 50 MG/ML / Potassium Chloride 0.02 MEQ/ML / Sodium Chloride 0.0769 MEQ/ML Injectable Solution	19123351
    # Glucose 50 MG/ML / Potassium Chloride 0.02 MEQ/ML / Sodium Chloride 0.154 MEQ/ML Injectable Solution	19123359
    # Glucose 50 MG/ML / Potassium Chloride 0.04 MEQ/ML / Sodium Chloride 0.0769 MEQ/ML Injectable Solution	19123353
    # Glucose 50 MG/ML / Potassium Chloride 0.04 MEQ/ML / Sodium Chloride 0.154 MEQ/ML Injectable Solution	968274
    # Glucose 50 MG/ML / Sodium Chloride 0.0342 MEQ/ML Injectable Solution	19124112
    # Glucose 50 MG/ML / Sodium Chloride 0.0769 MEQ/ML Injectable Solution	19076330
    # Glucose 50 MG/ML / Sodium Chloride 0.154 MEQ/ML Injectable Solution	967825


    #%% potassium
    # Alanine 10.4 MG/ML / Arginine 5.75 MG/ML / Calcium Chloride 0.004 MEQ/ML / Dibasic potassium phosphate 2.61 MG/ML / Glucose 150 MG/ML / Glycine 5.15 MG/ML / Histidine 2.4 MG/ML / Isoleucine 3 MG/ML / Leucine 3.65 MG/ML / Lysine 2.9 MG/ML / Magnesium Chlor	19131082
    # Alanine 8.8 MG/ML / Arginine 4.89 MG/ML / Calcium Chloride 0.004 MEQ/ML / Dibasic potassium phosphate 2.61 MG/ML / Glucose 50 MG/ML / Glycine 4.38 MG/ML / Histidine 2.04 MG/ML / Isoleucine 2.55 MG/ML / Leucine 3.11 MG/ML / Lysine 2.47 MG/ML / Magnesium Ch	19131065
    # Ascorbic Acid 4.7 MG/ML / POLYETHYLENE GLYCOL 3350 100 MG/ML / Potassium Chloride 0.0136 MEQ/ML / Sodium Ascorbate 5.9 MG/ML / Sodium Chloride 0.046 MEQ/ML / sodium sulfate 7.5 MG/ML Oral Solution [MoviPrep]	19131009
    # Calcium Chloride 0.0014 MEQ/ML / Glucose 50 MG/ML / Potassium Chloride 0.024 MEQ/ML / Sodium Chloride 0.103 MEQ/ML / Sodium Lactate 0.028 MEQ/ML Injectable Solution	968984
    # Calcium Chloride 0.0014 MEQ/ML / Potassium Chloride 0.004 MEQ/ML / Sodium Chloride 0.103 MEQ/ML / Sodium Lactate 0.028 MEQ/ML Injectable Solution	19135374
    # Calcium Chloride 0.001 MEQ/ML / Glucose 50 MG/ML / Potassium Chloride 0.004 MEQ/ML / Sodium Chloride 0.103 MEQ/ML / Sodium Lactate 0.028 MEQ/ML Injectable Solution	968985
    # Calcium Chloride 0.004 MEQ/ML / Magnesium Chloride 0.003 MEQ/ML / Potassium Chloride 0.01 MEQ/ML / Sodium Acetate 0.0475 MEQ/ML / Sodium Chloride 0.109 MEQ/ML / sodium citrate 0.012 MEQ/ML Ophthalmic Solution [BSS Ophthalmic Solution]	19132042
    # Clorazepate Dipotassium 3.75 MG Oral Tablet	790255
    # Dibasic potassium phosphate 1.66 MEQ/ML / Monobasic potassium phosphate 1.65 MEQ/ML Injectable Solution	40242057
    # Glucose 50 MG/ML / Potassium Chloride 0.02 MEQ/ML Injectable Solution	19076326
    # Glucose 50 MG/ML / Potassium Chloride 0.02 MEQ/ML / Sodium Chloride 0.0769 MEQ/ML Injectable Solution	19123351
    # Glucose 50 MG/ML / Potassium Chloride 0.02 MEQ/ML / Sodium Chloride 0.154 MEQ/ML Injectable Solution	19123359
    # Glucose 50 MG/ML / Potassium Chloride 0.04 MEQ/ML / Sodium Chloride 0.0769 MEQ/ML Injectable Solution	19123353
    # Glucose 50 MG/ML / Potassium Chloride 0.04 MEQ/ML / Sodium Chloride 0.154 MEQ/ML Injectable Solution	968274
    # Iodine 50 MG/ML / Potassium Iodide 100 MG/ML Oral Solution	19028377
    # Losartan Potassium 50 MG Oral Tablet	40185304
    # Magnesium Chloride 0.00148 MEQ/ML / Potassium Chloride 0.00497 MEQ/ML / Sodium Acetate 0.027 MEQ/ML / Sodium Chloride 0.0899 MEQ/ML / Sodium gluconate 5.02 MG/ML Injectable Solution [Plasmalyte A]	19131118
    # Penicillin G Potassium 1000000 UNT/ML Injectable Solution	40166455
    # Penicillin G Potassium 1000000 UNT/ML Injectable Solution [Pfizerpen]	40222954
    # Penicillin G Potassium 20000 UNT/ML Injectable Solution	19035413
    # Penicillin G Potassium 40000 UNT/ML Injectable Solution	19027561
    # Penicillin G Potassium 60000 UNT/ML Injectable Solution	19035414
    # Penicillin V Potassium 250 MG Oral Tablet	19133873
    # Penicillin V Potassium 500 MG Oral Tablet	19133905
    # POLYETHYLENE GLYCOL 3350 59 MG/ML / Potassium Chloride 0.01 MEQ/ML / Sodium Bicarbonate 0.02 MEQ/ML / Sodium Chloride 0.025 MEQ/ML / sodium sulfate 0.04 MEQ/ML Oral Solution [Golytely]	40175291
    # Potassium Acetate 2 MEQ/ML Injectable Solution	19076415
    # Potassium Chloride 0.02 MEQ/ML / Sodium Chloride 0.154 MEQ/ML Injectable Solution	968304
    # Potassium Chloride 0.1 MEQ/ML Injectable Solution	19079322
    # Potassium Chloride 0.2 MEQ/ML Injectable Solution	19049334
    # Potassium Chloride 0.4 MEQ/ML Injectable Solution	19049340
    # Potassium Chloride 10 MEQ Extended Release Oral Capsule	19079300
    # Potassium Chloride 10 MEQ Extended Release Oral Tablet [Klor-Con]	19049686
    # Potassium Chloride 1.33 MEQ/ML Oral Solution	19049335
    # Potassium Chloride 2 MEQ/ML Injectable Solution	19027661
    # potassium citrate 10 MEQ Extended Release Oral Tablet [Urocit-K]	19122549
    # Potassium Iodide 1000 MG/ML Oral Solution [SSKI brand of Potassium Iodide]	19114515
    # potassium nitrate 250 MG/ML / Silver Nitrate 750 MG/ML Medicated Pad	40242344
    # potassium phosphate 155 MG / Sodium Phosphate, Dibasic 852 MG / Sodium Phosphate, Monobasic 130 MG Oral Tablet [Phospha 250 Neutral]	991764


    #%% docusate
    # Docusate Sodium 100 MG Oral Capsule	40240688
    # Docusate Sodium 100 MG Oral Capsule [DOK]	44784779
    # Docusate Sodium 10 MG/ML Oral Suspension	42708743

    #%% heparin 
    # 250 ML heparin sodium, porcine 100 UNT/ML Injection	46275400
    # 3 ML heparin sodium, porcine 10 UNT/ML Prefilled Syringe	43011889
    # 500 ML heparin sodium, porcine 2 UNT/ML Injection	46275378
    # 5 ML heparin sodium, porcine 100 UNT/ML Prefilled Syringe	43011897
    # heparin sodium, porcine 10000 UNT/ML Injectable Solution	43011962
    # heparin sodium, porcine 1000 UNT/ML Injectable Solution	43011826
    # heparin sodium, porcine 10 UNT/ML Injectable Solution	43011809
    # heparin sodium, porcine 5000 UNT/ML Injectable Solution	43011850

    #%% magnesium_sulfate
    # 100 ML Magnesium Sulfate 40 MG/ML Injection	46275282
    # 20 ML Magnesium Sulfate 500 MG/ML Injection	46275266
    # 2 ML Magnesium Sulfate 500 MG/ML Injection	46275258
    # 500 ML Magnesium Sulfate 40 MG/ML Injection	46275283
    # 50 ML Magnesium Sulfate 40 MG/ML Injection	46275280
    # 50 ML Magnesium Sulfate 500 MG/ML Injection	46275274
    # Magnesium Sulfate 1000 MG Oral Granules	40234637

    #%% acetaminophen
    # Acetaminophen 100 MG/ML Oral Solution	1126750
    # Acetaminophen 100 MG/ML Oral Suspension [Mapap]	40231897
    # Acetaminophen 10 MG/ML Injectable Solution [Ofirmev]	40229093
    # Acetaminophen 120 MG Rectal Suppository	1125357
    # Acetaminophen 24 MG/ML / Codeine Phosphate 2.4 MG/ML Oral Solution	40221966
    # Acetaminophen 300 MG / Codeine Phosphate 30 MG Oral Tablet	40221904
    # Acetaminophen 325 MG / butalbital 50 MG / Caffeine 40 MG Oral Tablet	1134771
    # Acetaminophen 325 MG / Oxycodone Hydrochloride 5 MG Oral Tablet	40231925
    # Acetaminophen 325 MG / Oxycodone Hydrochloride 5 MG Oral Tablet [Roxicet]	40231928
    # Acetaminophen 32 MG/ML Oral Solution	19072176
    # Acetaminophen 500 MG / Hydrocodone Bitartrate 5 MG Oral Tablet	40162494
    # Acetaminophen 500 MG Oral Tablet	19020053
    # Acetaminophen 500 MG Oral Tablet [Mapap]	19041464
    # Acetaminophen 650 MG Rectal Suppository	1125392
    # Acetaminophen 650 MG Rectal Suppository [Acephen]	19041255
    # Acetaminophen 65 MG/ML / Oxycodone Hydrochloride 1 MG/ML Oral Solution [Roxicet]	40231944
    # Acetaminophen 80 MG Rectal Suppository [Feverall]	19041658


    #%% pantoprazole
    # pantoprazole 40 MG Delayed Release Oral Tablet [Protonix]	19071430
    # pantoprazole 4 MG/ML Injectable Solution [Protonix]	40168991

    #%% metoprolol
    # 24 HR metoprolol succinate 100 MG Extended Release Oral Tablet [Toprol]	40166825
    # 24 HR metoprolol succinate 25 MG Extended Release Oral Tablet	40166828
    # 24 HR metoprolol succinate 25 MG Extended Release Oral Tablet [Toprol]	40166829
    # 24 HR metoprolol succinate 50 MG Extended Release Oral Tablet	40166830
    # 24 HR metoprolol succinate 50 MG Extended Release Oral Tablet [Toprol]	40166831
    # Metoprolol Tartrate 100 MG Oral Tablet	40167202
    # Metoprolol Tartrate 1 MG/ML Injectable Solution	40167196
    # Metoprolol Tartrate 25 MG Oral Tablet	40167213
    # Metoprolol Tartrate 50 MG Oral Tablet	40167218

    #%% furosemide
    # Furosemide 10 MG/ML Injectable Solution	957134
    # Furosemide 10 MG/ML Oral Solution	19019042
    # Furosemide 20 MG Oral Tablet	19077546
    # Furosemide 40 MG Oral Tablet	957136
    # Furosemide 80 MG Oral Tablet	956877
    # Furosemide 8 MG/ML Oral Solution	956876

    primitives["medications"] = {"table":"drug_exposure", "column":"drug_concept_id", "count_distinct":"drug_exposure_id",
                          "categories":["sodium_chloride", "glucose", "potassium", "docusate", "heparin", "magnesium_sulfate", 
                                       "acetaminophen", "pantoprazole", "metoprolol", "furosemide"],

                          "sql_values":[[19132042, 19123359, 19076330, 19131118, 968304, 968984, 19131009, 19124112, 19079524,
                                         40175291, 19123351, 19079551, 967825, 19135374, 19128117, 968205, 19131053, 967905, 19127417, 19079527,
                                         19127005, 19123353, 968985,968274],

                                        [19123359, 19076330, 968984, 19124112, 19131065, 19077654, 19096125, 19123351, 1560554, 967825, 1560751, 19076324, 19131082, 43531812, 19123353, 19076326, 968985, 968274, 1560553],

                                        [19132042, 19123359, 19049334, 19035413, 19079322, 19131118, 40185304, 968304, 19114515,
                                         968984, 19049335, 19027561, 19131009, 19122549, 19079300, 19131065, 40175291, 19123351, 19049340,
                                         40242057, 40166455, 19135374,19131082, 991764, 19133873, 40242344, 19076415, 19035414, 19028377, 19027661,
                                         19133905, 19049686, 790255, 19123353, 19076326, 968985, 968274, 40222954],

                                        [44784779, 40240688, 42708743],

                                        [46275400, 43011889, 46275378, 43011897, 43011962, 43011826, 43011809, 43011850],

                                        [46275282, 46275266, 46275258, 46275283, 46275280, 46275274, 40234637],

                                        [19041464, 19041658, 19072176, 19041255, 40231944, 1134771, 40229093, 40221966, 
                                         1125357, 40231897, 19020053, 40231925, 1125392, 40221904, 40231928, 1126750, 40162494],

                                        [40168991, 19071430],

                                        [40166825, 40166825, 40166828, 40166829, 40166830, 40166831, 40167202, 40167196, 
                                         40167213, 40167218],

                                        [957134, 19019042, 19077546, 957136, 956877, 956876]]}

    primitives["medications"]["results"] = [[0] for x in primitives["medications"]["categories"]]
    primitives["medications"]["percent_eliminated"] = [[0] for x in primitives["medications"]["categories"]]
    
    return primitives

In [285]:
# need 2 sql queries that can be generic/parameterized
# 1. number of rows without any filtering in the table in question (maybe group by patient or visit id?)
# 2. number of matches per condition_sub condition (these need to be a list with one value per sub condition)
# 3. we might need 2 w/ and without groupby ..? 

def generate_paramterized_query(primitive_dict_entry, primitive):
     
    denominator_query = "SELECT COUNT(DISTINCT({})) FROM mimic_v5.{};".format(primitive_dict_entry['count_distinct'],            
                                                              primitive_dict_entry['table'])
    results_total_count = run_query(denominator_query, primitive_dict_entry)
    primitive_dict_entry['total_count'] = results_total_count
    
    counter = 0   
    
    for category in primitive_dict_entry['categories']:
        
        #vals = (x for x in primitive_dict_entry['sql_values'][counter])
        
        #print(len(primitive_dict_entry['sql_values'][counter]))
        
        vals = tuple(x for x in primitive_dict_entry['sql_values'][counter])
        
        if len(vals) == 1:
            vals = ('(\'{}\')'.format(vals[0]))
        
        
        numerator_query = "SELECT COUNT(DISTINCT({})) FROM mimic_v5.{} WHERE {} IN {};".format(primitive_dict_entry['count_distinct'],            
                                                              primitive_dict_entry['table'], 
                                                              primitive_dict_entry['column'], 
                                                              vals)
        
        
        
        results_subset = run_query(numerator_query, primitive, category)
        primitive_dict_entry['results'][counter] = results_subset 
        primitive_dict_entry['percent_eliminated'][counter] = (int(results_total_count)-int(results_subset))/int(results_total_count)
        print("Positive instances: {}; percent eliminated: {}".format(results_subset, primitive_dict_entry['percent_eliminated'][counter]))
        
        counter += 1
        
#generate_paramterized_query(primitives["gender"], "gender")

In [286]:
def rank_primitives_by_downselection_potential(primitives):
    
    num_items = 0
    for k,v in primitives.items():
        print(k,v)
        num_items += len(primitives[k]['categories'])

    rank_df = pd.DataFrame(columns=["primitive", "category", "count", "percent_eliminated", "downselection_rank"], 
                           index=[i for i in range(num_items)])
    
    start_index = 0
    for k, v in primitives.items():
        num_rows = len(primitives[k]['categories'])
        rank_df.loc[start_index:start_index+num_rows, 'primitive'] = k
        
        for j in range(num_rows):
            rank_df.loc[start_index+j, 'category'] = primitives[k]["categories"][j]
            rank_df.loc[start_index+j, 'count'] = primitives[k]['results'][j]
            rank_df.loc[start_index+j, 'percent_eliminated'] = primitives[k]['percent_eliminated'][j]
        
        start_index += num_rows
    
    rank_df = rank_df.sort_values(['percent_eliminated'], ascending=False).reset_index(drop=True)
    rank_df['downselection_rank'] = rank_df.index

    return rank_df          
    


In [287]:
def main():
    
    primitives = generate_primitives_dict()

    for key, value in primitives.items():
        generate_paramterized_query(primitives[key], key)
        
    return rank_primitives_by_downselection_potential(primitives)

out = main()  


Positive instances: 26121; percent eliminated: 0.4385116399045592
Positive instances: 20399; percent eliminated: 0.5615098557640635
Positive instances: 1691; percent eliminated: 0.9636508243588917
Positive instances: 3875; percent eliminated: 0.9167042840867565
Positive instances: 47; percent eliminated: 0.9989897035747297
Positive instances: 0; percent eliminated: 1.0
Positive instances: 33899; percent eliminated: 0.27131832935663464
Positive instances: 22779; percent eliminated: 0.9594393914573258
Positive instances: 107; percent eliminated: 0.9998094742914937
Positive instances: 14680; percent eliminated: 0.9738605850385681
Positive instances: 15796; percent eliminated: 0.9718734197049879
Positive instances: 16678; percent eliminated: 0.9703029180703842
Positive instances: 14627; percent eliminated: 0.9739549575857722
Positive instances: 6578; percent eliminated: 0.988287120462105
Positive instances: 6326; percent eliminated: 0.9887358352148489
Positive instances: 14541; percent eli

In [288]:
out

,primitive,category,count,percent_eliminated,downselection_rank
0,race,pacific_islander,0,1,0
1,conditions,chf,107,0.999809,1
2,race,native_american,47,0.99899,2
3,medications,pantoprazole,46089,0.989282,3
4,conditions,gerd,6326,0.988736,4
5,conditions,uti,6578,0.988287,5
6,medications,docusate,52667,0.987752,6
7,medications,acetaminophen,54257,0.987382,7
8,conditions,respiratory_failure,8524,0.984822,8
9,medications,magnesium_sulfate,80516,0.981276,9


In [289]:
out.to_csv("./inclusion_criteria_ranked_by_downselection.csv")